In [1]:
from utils.metrics import metric
from data_provider.data_loader import Dataset_Custom
from models import xPatch
from utils.tools import EarlyStopping, adjust_learning_rate, visual
from data_provider.data_factory import data_provider
from exp.exp_main import Exp_Main
import sys
import os
import time
import warnings
import math
import argparse
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

project_root = os.path.abspath('./')
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
# Complete sweep cell for notebook
import wandb


def notebook_sweep():
    # Advanced sweep config
    sweep_config = {
        'method': 'bayes',
        'metric': {
            'name': 'final_test_mse',
            'goal': 'minimize'
        },
        'parameters': {
            # Learning optimization
            'learning_rate': {
                'distribution': 'log_uniform_values',
                'min': 0.00001,
                'max': 0.001
            },
            'batch_size': {'values': [8, 16, 32]},  # Reduced range
            # Fixed: should be a parameter, not fixed value
            'train_epochs': {'values': [3, 5, 8]},

            # Model architecture - xPatch specific
            'd_model': {'values': [64, 128, 256]},  # Reduced range
            # Fixed values instead of distribution
            'd_ff': {'values': [128, 256, 512, 768]},
            'e_layers': {'values': [2, 3, 4]},  # Reduced range
            'dropout': {
                'distribution': 'uniform',
                'min': 0.1,
                'max': 0.3
            },

            # Patching strategy - MORE CONSERVATIVE
            # Reduced range, avoiding very small/large
            'patch_len': {'values': [8, 12, 16]},
            'stride': {'values': [4, 6, 8]},  # Reduced range

            # Sequence modeling - CONSERVATIVE
            'seq_len': {'values': [48, 72, 96]},  # Reduced range
            'pred_len': {'values': [3, 6, 9]},    # Full range

            # Moving average parameters
            'ma_type': {'values': ['ema', 'dema']},
            'alpha': {
                'distribution': 'uniform',
                'min': 0.1,
                'max': 0.4
            },
            'beta': {
                'distribution': 'uniform',
                'min': 0.1,
                'max': 0.4
            },

            # Advanced xPatch parameters
            'k': {'values': [2, 3, 4]},  # Reduced range
            'decomp': {'values': [0, 1]},

            # Learning rate scheduling
            # Removed 'cosine' temporarily
            'lradj': {'values': ['type1', 'type2']},

            # Regularization
            'revin': {'values': [0, 1]},  # Reduced range

            # LSTM Configuration (if using LSTM enhanced model)
            'use_lstm': {'values': [True, False]},
            # Integer values
            'lstm_hidden_size': {'values': [64, 128, 192, 256]},
            'lstm_layers': {'values': [1, 2, 3, 4]},  # Integer values
            'lstm_dropout': {
                'distribution': 'uniform',
                'min': 0.1,
                'max': 0.3
            },

        },

        # Advanced early termination
        'early_terminate': {
            'type': 'hyperband',
            'min_iter': 2,
            'max_iter': 15,
            'eta': 3,
            's': 2
        }
    }

    # Create sweep
    sweep_id = wandb.sweep(sweep_config,
                           project="CS7643-GroupProject",
                           entity="xplstm")

    def train_run():
        # Define Args class inside the function so it's available
        class Args:
            def __init__(self):
                self.is_training = 1
                self.train_only = False
                self.model_id = 'AAPL_Pred5_Notebook_Tuned'
                self.model = 'xPatch'
                self.data = 'custom'  # Fixed: should be 'custom' for AAPL dataset
                self.scale = True
                self.root_path = './data/'
                self.data_path = 'aapl_OHLCV.csv'
                self.features = 'MS'
                self.target = 'Close'  # Fixed: should be 'Close' for AAPL dataset
                self.freq = 'd'
                self.checkpoints = './checkpoints/'
                self.embed = 'timeF'
                self.label_len = 48
                self.enc_in = 9  # Fixed: AAPL dataset actually has 9 features including time features
                self.padding_patch = 'end'
                self.num_workers = 8
                self.itr = 1
                self.patience = 15
                self.des = 'Sweep_Run'
                self.loss = 'mae'
                self.revin = 1

                # Default values (will be overridden by sweep)
                self.seq_len = 48
                self.pred_len = 5
                self.patch_len = 12
                self.stride = 6
                self.ma_type = 'ema'
                self.alpha = 0.2
                self.beta = 0.2
                self.train_epochs = 3
                self.batch_size = 16
                self.learning_rate = 0.0001
                self.lradj = 'type1'
                self.d_model = 128
                self.d_ff = 256
                self.e_layers = 3
                self.dropout = 0.1
                self.k = 3
                self.decomp = 0

                # LSTM Configuration (if using LSTM enhanced model)
                self.use_lstm = True
                self.lstm_hidden_size = 192
                self.lstm_layers = 4
                self.lstm_dropout = 0.2
                self.lstm_bidirectional = True

                # GPU settings
                self.use_gpu = True if torch.cuda.is_available() else False
                self.gpu = 0
                self.use_multi_gpu = False
                self.devices = '0'
                self.test_flop = False

                # Wandb settings
                self.use_wandb = True
                self.wandb_entity = 'xplstm'
                self.wandb_project = 'CS7643-GroupProject'
                self.experiment_notes = 'Sweep experiment'

        # Initialize wandb run
        run = wandb.init()

        # Check if wandb initialization was successful
        if run is None:
            print("Failed to initialize wandb run")
            return

        try:
            # Create Args instance
            args = Args()

            # Override all sweep parameters with proper type conversion
            for param, value in wandb.config.items():
                if hasattr(args, param):
                    try:
                        # Special handling for parameters that need to be integers
                        if param in ['lstm_hidden_size', 'lstm_layers', 'train_epochs', 'batch_size',
                                     'seq_len', 'pred_len', 'patch_len', 'stride', 'd_model', 'd_ff', 'e_layers', 'k']:
                            setattr(args, param, int(float(value)))  # Convert via float first to handle numpy types
                        # Special handling for boolean parameters
                        elif param in ['use_lstm', 'lstm_bidirectional', 'revin', 'decomp']:
                            # Handle various boolean representations
                            if isinstance(value, (bool, int)):
                                setattr(args, param, bool(value))
                            elif isinstance(value, str):
                                setattr(args, param, value.lower() in ['true', '1', 'yes'])
                            else:
                                setattr(args, param, bool(value))
                        # Special handling for float parameters
                        elif param in ['learning_rate', 'dropout', 'alpha', 'beta', 'lstm_dropout']:
                            setattr(args, param, float(value))
                        else:
                            setattr(args, param, value)
                    except (ValueError, TypeError) as e:
                        print(f"Warning: Could not convert parameter {param}={value}: {e}")
                        # Use default value or skip
                        continue

            # Ensure model_id is unique - use the stored run reference
            args.model_id = f"sweep_{run.name}"

            # COMPREHENSIVE TYPE CONVERSION: Ensure ALL parameters are native Python types
            # Convert all integer parameters
            int_params = ['lstm_hidden_size', 'lstm_layers', 'train_epochs', 'batch_size',
                         'seq_len', 'pred_len', 'patch_len', 'stride', 'd_model', 'd_ff', 'e_layers', 'k']
            for param in int_params:
                if hasattr(args, param):
                    setattr(args, param, int(float(str(getattr(args, param)))))
            
            # Convert all float parameters
            float_params = ['learning_rate', 'dropout', 'alpha', 'beta', 'lstm_dropout']
            for param in float_params:
                if hasattr(args, param):
                    setattr(args, param, float(str(getattr(args, param))))
            
            # Convert all boolean parameters
            bool_params = ['use_lstm', 'lstm_bidirectional', 'revin', 'decomp']
            for param in bool_params:
                if hasattr(args, param):
                    val = getattr(args, param)
                    if isinstance(val, (int, float)):
                        setattr(args, param, bool(int(val)))
                    elif isinstance(val, str):
                        setattr(args, param, val.lower() in ['true', '1', 'yes'])
                    else:
                        setattr(args, param, bool(val))

            # CRITICAL: Enhanced parameter validation and adjustment
            # 1. Ensure patch_len is reasonable for seq_len
            if args.patch_len > args.seq_len:
                args.patch_len = args.seq_len // 4
                wandb.log(
                    {"param_adjustment": f"patch_len adjusted to {args.patch_len}"})

            # 2. Ensure stride is reasonable for patch_len
            if args.stride > args.patch_len:
                args.stride = max(1, args.patch_len // 2)
                wandb.log(
                    {"param_adjustment": f"stride adjusted to {args.stride}"})

            # 3. Calculate number of patches and ensure it's > 0
            num_patches = max(
                1, (args.seq_len - args.patch_len) // args.stride + 1)
            if num_patches <= 0:
                # Adjust parameters to ensure at least 1 patch
                args.patch_len = min(args.patch_len, args.seq_len)
                args.stride = min(args.stride, args.patch_len)
                num_patches = max(
                    1, (args.seq_len - args.patch_len) // args.stride + 1)
                wandb.log({
                    "param_adjustment": f"Fixed patching: patch_len={args.patch_len}, stride={args.stride}, num_patches={num_patches}"
                })

            # 4. Ensure d_ff >= d_model
            if args.d_ff < args.d_model:
                args.d_ff = args.d_model * 2
                wandb.log({"param_adjustment": f"d_ff adjusted to {args.d_ff}"})

            # 5. Ensure pred_len < seq_len
            if args.pred_len >= args.seq_len:
                args.pred_len = min(args.pred_len, args.seq_len // 4)
                wandb.log(
                    {"param_adjustment": f"pred_len adjusted to {args.pred_len}"})

            # 6. Additional validation for xPatch specific parameters
            if args.patch_len < 4:
                args.patch_len = 4
                wandb.log(
                    {"param_adjustment": f"patch_len minimum adjusted to {args.patch_len}"})

            if args.stride < 1:
                args.stride = 1
                wandb.log(
                    {"param_adjustment": f"stride minimum adjusted to {args.stride}"})

            # 7. Validate the final configuration
            final_num_patches = (
                args.seq_len - args.patch_len) // args.stride + 1
            if final_num_patches <= 0:
                raise ValueError(f"Invalid configuration results in {final_num_patches} patches. "
                                 f"seq_len={args.seq_len}, patch_len={args.patch_len}, stride={args.stride}")

            # 8. Ensure LSTM parameters are valid if LSTM is enabled with extensive debugging
            if hasattr(args, 'use_lstm') and args.use_lstm:
                # Set defaults if missing
                if not hasattr(args, 'lstm_hidden_size'):
                    args.lstm_hidden_size = 192
                if not hasattr(args, 'lstm_layers'):
                    args.lstm_layers = 2
                if not hasattr(args, 'lstm_dropout'):
                    args.lstm_dropout = 0.2
                if not hasattr(args, 'lstm_bidirectional'):
                    args.lstm_bidirectional = True

                # CRITICAL: Convert to native Python types to avoid PyTorch tensor issues
                # This prevents "'float' object has no attribute 'to'" errors
                args.lstm_hidden_size = int(float(str(args.lstm_hidden_size)))
                args.lstm_layers = int(float(str(args.lstm_layers)))
                args.lstm_dropout = float(str(args.lstm_dropout))
                
                # Special handling for boolean conversion
                if isinstance(args.lstm_bidirectional, (int, float)):
                    args.lstm_bidirectional = bool(int(args.lstm_bidirectional))
                elif isinstance(args.lstm_bidirectional, str):
                    args.lstm_bidirectional = args.lstm_bidirectional.lower() in ['true', '1', 'yes']
                else:
                    args.lstm_bidirectional = bool(args.lstm_bidirectional)

                # Validate ranges
                args.lstm_hidden_size = max(1, min(512, args.lstm_hidden_size))
                args.lstm_layers = max(1, min(8, args.lstm_layers))
                args.lstm_dropout = max(0.0, min(0.9, args.lstm_dropout))

                # Debug logging
                print(f"LSTM Configuration (converted to native Python types):")
                print(f"  lstm_hidden_size: {args.lstm_hidden_size} (type: {type(args.lstm_hidden_size)})")
                print(f"  lstm_layers: {args.lstm_layers} (type: {type(args.lstm_layers)})")
                print(f"  lstm_dropout: {args.lstm_dropout} (type: {type(args.lstm_dropout)})")
                print(f"  lstm_bidirectional: {args.lstm_bidirectional} (type: {type(args.lstm_bidirectional)})")

                wandb.log({
                    "config/lstm_enabled": True,
                    "config/lstm_hidden_size": int(args.lstm_hidden_size),
                    "config/lstm_layers": int(args.lstm_layers),
                    "config/lstm_dropout": float(args.lstm_dropout),
                    "config/lstm_bidirectional": bool(args.lstm_bidirectional)
                })

            # Log parameter combinations for analysis
            wandb.log({
                "config/patch_ratio": args.patch_len / args.seq_len,
                "config/stride_ratio": args.stride / args.patch_len,
                "config/model_complexity": args.d_model * args.e_layers,
                "config/ff_ratio": args.d_ff / args.d_model,
                "config/prediction_horizon": args.pred_len / args.seq_len,
                "config/num_patches": final_num_patches,
                "config/patch_coverage": (args.patch_len + (final_num_patches - 1) * args.stride) / args.seq_len
            })

            # Add error handling for configuration issues
            if args.patch_len <= 0 or args.stride <= 0:
                raise ValueError(
                    f"Invalid patch configuration: patch_len={args.patch_len}, stride={args.stride}")

            if args.d_model <= 0 or args.d_ff <= 0:
                raise ValueError(
                    f"Invalid model dimensions: d_model={args.d_model}, d_ff={args.d_ff}")

            if not os.path.exists(args.checkpoints):
                os.makedirs(args.checkpoints)

            print(f"Starting sweep run: {args.model_id}")
            print(f"Key parameters: lr={args.learning_rate:.6f}, batch={args.batch_size}, "
                  f"patch_len={args.patch_len}, d_model={args.d_model}")
            print(f"Patching info: seq_len={args.seq_len}, patch_len={args.patch_len}, "
                  f"stride={args.stride}, num_patches={final_num_patches}")

            # Additional debugging for all numeric parameters
            print("All numeric parameter types:")
            numeric_params = ['seq_len', 'pred_len', 'patch_len', 'stride', 'd_model', 'd_ff', 'e_layers', 
                            'batch_size', 'train_epochs', 'learning_rate', 'dropout', 'k', 'decomp', 'revin']
            for param in numeric_params:
                if hasattr(args, param):
                    val = getattr(args, param)
                    print(f"  {param}: {val} (type: {type(val)})")

            # Create experiment and train
            try:
                print("Creating Exp_Main instance...")
                exp = Exp_Main(args)
                print("Exp_Main created successfully")
                
                print("Starting training...")
                best_model = exp.train(args.model_id)
                print("Training completed successfully")
                
                print("Starting testing...")
                exp.test(args.model_id)
                print("Testing completed successfully")
            except Exception as model_error:
                print(f"Error during model creation/training: {str(model_error)}")
                print(f"Error type: {type(model_error)}")
                import traceback
                print("Full traceback:")
                traceback.print_exc()
                raise  # Re-raise to see the full error

            # Log success
            if run is not None:
                wandb.log({"run_status": "completed"})

        except Exception as e:
            print(f"Error in sweep run: {str(e)}")
            # More robust error logging
            try:
                if run is not None:
                    wandb.log({
                        "run_status": "failed",
                        "error_message": str(e),
                        # Ensure bad runs are penalized
                        "final_test_mse": float('inf')
                    })
            except:
                print("Could not log error to wandb")

            # Don't re-raise the exception - just let the run fail gracefully
            return

        finally:
            # Ensure wandb run is properly finished
            if run is not None:
                wandb.finish()

    print(
        f"Sweep URL: https://wandb.ai/xplstm/CS7643-GroupProject/sweeps/{sweep_id}")
    print("Advanced sweep configuration:")
    print(f"- {len(sweep_config['parameters'])} hyperparameters")
    print(f"- Bayesian optimization with Hyperband early termination")
    print(f"- Parameter validation and auto-adjustment")

    # Run sweep with more experiments for comprehensive search
    wandb.agent(sweep_id, function=train_run, count=3)

    return sweep_id


# Uncomment to run advanced sweep
sweep_id = notebook_sweep()

Create sweep with ID: czecy325
Sweep URL: https://wandb.ai/xplstm/CS7643-GroupProject/sweeps/czecy325
Sweep URL: https://wandb.ai/xplstm/CS7643-GroupProject/sweeps/czecy325
Advanced sweep configuration:
- 22 hyperparameters
- Bayesian optimization with Hyperband early termination
- Parameter validation and auto-adjustment


wandb: Agent Starting Run: quyyiwlb with config:
wandb: 	alpha: 0.2790763105617088
wandb: 	batch_size: 8
wandb: 	beta: 0.2673467177823954
wandb: 	d_ff: 128
wandb: 	d_model: 128
wandb: 	decomp: 1
wandb: 	dropout: 0.1479033708191273
wandb: 	e_layers: 2
wandb: 	k: 2
wandb: 	learning_rate: 0.00035472069395483787
wandb: 	lradj: type2
wandb: 	lstm_dropout: 0.1020361095893809
wandb: 	lstm_hidden_size: 64
wandb: 	lstm_layers: 2
wandb: 	ma_type: ema
wandb: 	patch_len: 8
wandb: 	pred_len: 6
wandb: 	revin: 1
wandb: 	seq_len: 48
wandb: 	stride: 8
wandb: 	train_epochs: 5
wandb: 	use_lstm: True
wandb: Currently logged in as: radeosimbio (xplstm) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


LSTM Configuration (converted to native Python types):
  lstm_hidden_size: 64 (type: <class 'int'>)
  lstm_layers: 2 (type: <class 'int'>)
  lstm_dropout: 0.1020361095893809 (type: <class 'float'>)
  lstm_bidirectional: True (type: <class 'bool'>)
Starting sweep run: sweep_spring-sweep-1
Key parameters: lr=0.000355, batch=8, patch_len=8, d_model=128
Patching info: seq_len=48, patch_len=8, stride=8, num_patches=6
All numeric parameter types:
  seq_len: 48 (type: <class 'int'>)
  pred_len: 6 (type: <class 'int'>)
  patch_len: 8 (type: <class 'int'>)
  stride: 8 (type: <class 'int'>)
  d_model: 128 (type: <class 'int'>)
  d_ff: 128 (type: <class 'int'>)
  e_layers: 2 (type: <class 'int'>)
  batch_size: 8 (type: <class 'int'>)
  train_epochs: 5 (type: <class 'int'>)
  learning_rate: 0.00035472069395483787 (type: <class 'float'>)
  dropout: 0.1479033708191273 (type: <class 'float'>)
  k: 2 (type: <class 'int'>)
  decomp: True (type: <class 'bool'>)
  revin: True (type: <class 'bool'>)
Creat

config/ff_ratio,▁
config/lstm_dropout,▁
config/lstm_hidden_size,▁
config/lstm_layers,▁
config/model_complexity,▁
config/num_patches,▁
config/patch_coverage,▁
config/patch_ratio,▁
config/prediction_horizon,▁
config/stride_ratio,▁
config/ff_ratio,1


Exp_Main created successfully
Starting training...
train 2665
val 385
test 771
	iters: 100, epoch: 1 | loss: 0.0815917
	speed: 0.0667s/iter; left time: 104.4362s
	iters: 200, epoch: 1 | loss: 0.0326682
	speed: 0.0132s/iter; left time: 19.4136s
	iters: 300, epoch: 1 | loss: 0.0120203
	speed: 0.0130s/iter; left time: 17.7315s
Epoch: 1 cost time: 6.233487606048584
Epoch: 1, Steps: 333 | Train Loss: 0.0352412 Vali Loss: 0.0939851 Test Loss: 0.0924085
Validation loss decreased (inf --> 0.093985).  Saving model ...
	iters: 100, epoch: 2 | loss: 0.0220940
	speed: 0.0274s/iter; left time: 33.7525s
	iters: 200, epoch: 2 | loss: 0.0093081
	speed: 0.0119s/iter; left time: 13.5369s
	iters: 300, epoch: 2 | loss: 0.0200510
	speed: 0.0118s/iter; left time: 12.1517s
Epoch: 2 cost time: 4.2413835525512695
Epoch: 2, Steps: 333 | Train Loss: 0.0195322 Vali Loss: 0.0876950 Test Loss: 0.0838167
Validation loss decreased (0.093985 --> 0.087695).  Saving model ...
Updating learning rate to 5e-05
	iters: 100,

...

LSTM Configuration (converted to native Python types):
  lstm_hidden_size: 64 (type: <class 'int'>)
  lstm_layers: 1 (type: <class 'int'>)
  lstm_dropout: 0.17221356555853634 (type: <class 'float'>)
  lstm_bidirectional: True (type: <class 'bool'>)
Starting sweep run: sweep_resilient-sweep-3
Key parameters: lr=0.000040, batch=8, patch_len=8, d_model=128
Patching info: seq_len=96, patch_len=8, stride=6, num_patches=15
All numeric parameter types:
  seq_len: 96 (type: <class 'int'>)
  pred_len: 3 (type: <class 'int'>)
  patch_len: 8 (type: <class 'int'>)
  stride: 6 (type: <class 'int'>)
  d_model: 128 (type: <class 'int'>)
  d_ff: 256 (type: <class 'int'>)
  e_layers: 2 (type: <class 'int'>)
  batch_size: 8 (type: <class 'int'>)
  train_epochs: 8 (type: <class 'int'>)
  learning_rate: 3.9957954173606464e-05 (type: <class 'float'>)
  dropout: 0.11163913662469065 (type: <class 'float'>)
  k: 3 (type: <class 'int'>)
  decomp: False (type: <class 'bool'>)
  revin: True (type: <class 'bool'>

config/ff_ratio,▁
config/lstm_dropout,▁
config/lstm_hidden_size,▁
config/lstm_layers,▁
config/model_complexity,▁
config/num_patches,▁
config/patch_coverage,▁
config/patch_ratio,▁
config/prediction_horizon,▁
config/stride_ratio,▁
config/ff_ratio,2


Exp_Main created successfully
Starting training...
train 2620
val 388
test 774
	iters: 100, epoch: 1 | loss: 0.0658615
	speed: 0.0120s/iter; left time: 30.1812s
	iters: 200, epoch: 1 | loss: 0.0966382
	speed: 0.0108s/iter; left time: 26.1695s
	iters: 300, epoch: 1 | loss: 0.1447263
	speed: 0.0111s/iter; left time: 25.6629s
Epoch: 1 cost time: 3.7057924270629883
Epoch: 1, Steps: 327 | Train Loss: 0.0974867 Vali Loss: 0.2920392 Test Loss: 0.4387103
Validation loss decreased (inf --> 0.292039).  Saving model ...
	iters: 100, epoch: 2 | loss: 0.0780010
	speed: 0.0250s/iter; left time: 54.6868s
	iters: 200, epoch: 2 | loss: 0.0534791
	speed: 0.0110s/iter; left time: 22.9907s
	iters: 300, epoch: 2 | loss: 0.0919431
	speed: 0.0103s/iter; left time: 20.5239s
Epoch: 2 cost time: 3.6732540130615234
Epoch: 2, Steps: 327 | Train Loss: 0.0878092 Vali Loss: 0.2694064 Test Loss: 0.3954282
Validation loss decreased (0.292039 --> 0.269406).  Saving model ...
Updating learning rate to 5e-05
	iters: 100,

batch,▁▁▃▅▅█▂▃▇▇▂▄▅▆▃▇▇▁▄▅▆▇▇▁▂▄▅▆▇▁▃▅▆█▁▄▆▇▇█
batch_loss,▄▄▅▆█▃▄▃▆▃▄▅▂█▅▁▃▇▂▁▄▁▂▂▃▃▄▃▃▄▆▂▁▂▁▁▂▃▂▃
best_sample_mse,▁
epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇██████
epoch_time,▅▄▃█▇▃▁▄
final_test_mae,▁
final_test_mse,▁
learning_rate,▆▆▆▆▆▆▆▆▆▆▆▆████████████▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
mse_std,▁
test_loss,█▇▃▂▁▁▁▁
test_samples,▁▁


Testing completed successfully
Error in sweep run: You must call wandb.init() before wandb.log()
Could not log error to wandb
